Prior to definining our python3 environment, I will call pip manager to install 3rd-party modules:

In [0]:
#install any missing packages:
!pip install fuzzywuzzy
!pip install pydrive
!pip install python-Levenshtein

Import known modules and authenticate google credentials:

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#Load modules for our own analysis:
import pandas as pd
import numpy as np

#Initialise the google authentication:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


The following is a link to the Abodoo database (NOTE: this is a copy of the main file, and I have it converted to xlsx to minimise chances of incompatibility ocurring):

https://drive.google.comhttps://drive.google.com/open?id=1SRatK8tSCQKrb3x6G8iugPn7ZUxzb7H1

The following is a link to the TownsDataBase module which provides functionality for matching city/country placenames and assigning additional complementary datafields (population, location, class of city, and more.)

https://drive.google.com/open?id=15HXkcWFmCqOT3sfJCH41vckeiMBo9Dv3

Tow load these files, we use pyDrive, and the pass the ID's contained in eacho of the above links.



In [0]:


AbodooDB_file = drive.CreateFile({'id':'1SRatK8tSCQKrb3x6G8iugPn7ZUxzb7H1'})
AbodooDB_file.GetContentFile('AbodooDB.xlsx')
TownsDataBase_file = drive.CreateFile({'id':'15HXkcWFmCqOT3sfJCH41vckeiMBo9Dv3'})
TownsDataBase_file.GetContentFile('TownsDataBase.py')

Now we have loaded two files from google drive, let's get to work!

In [0]:
import TownsDataBase      #import our module

TownsDataBase.recompileDB() #recompile the TownsDB
townsDB = TownsDataBase.loadDB() #load the TownsDB into memory


Frm=pd.read_excel('AbodooDB.xlsx')   #open up the xlsx DB and load it as a pandas Frame


Unpacking Archive...
Unpacking Complete


In [0]:
CountryKey = []
CityKey = []
CountryMatchSucess = []
CityMatchSucess = []
Length=len(Frm.COUNTRY)
inDx=0
for country, city in zip(Frm.COUNTRY[:50], Frm.CITY[:50]):
  #
  if country == np.nan: country = None
  if city == np.nan: city = None
  print('query:{} of {} Country=\"{}\" and City=\"{}\"'.format(inDx,Length,country,city))
  #Query for country:
  resultCountryQuery = TownsDataBase.queryCountryName(country,townsDB)
  #Check matched country result, and save:
  if resultCountryQuery[0] != None:
    print('\t matched with countryID = {} ({}%)'.format(resultCountryQuery[0],100*(resultCountryQuery[1]/1.0)))
    CountryKey.append(resultCountryQuery[0])
    CountryMatchSucess.append(resultCountryQuery[1])
  else:
    print('\t Matching unsuccessful! no countryID found.)')
    CountryKey.append(np.nan)
    CountryMatchSucess.append(np.nan)
  #Query city:
  resultCityQuery = TownsDataBase.queryCityName(city,townsDB,resultCountryQuery[0])
  if resultCountryQuery[0] == None and resultCityQuery[3] != None:
    CountryKey[-1] = resultCityQuery[3] #if we find a city without country Key, update the new countryKey!
  if resultCityQuery[0] != None:
    print('\t\t matched with city = {} ({}%)'.format(resultCityQuery[0],100*(resultCityQuery[1]/1.0)))
    CityKey.append(resultCityQuery[2])
    CityMatchSucess.append(resultCityQuery[1])
  else:
    print('\t\t Matching unsuccessful! no cityID found.)')
    CityKey.append(np.nan)
    CityMatchSucess.append(np.nan)
  inDx += 1
  

  
print('\n\ncity Keys found:')
for keys1,keys2 in zip(CountryKey,CityKey):
  print('countryKey: {}, cityKey: {}'.format(keys1,keys2))
                                     

query:0 of 1610 Country="France" and City="Antibes"
	 matched with countryID = FR (100.0%)
		 matched with city = Antibes (100.0%)
query:1 of 1610 Country="United Arab Emirates" and City="Abu Dhabi"
	 matched with countryID = AE (100.0%)
		 matched with city = Abu Dhabi (100.0%)
query:2 of 1610 Country="Ireland" and City="Dublin"
	 matched with countryID = IE (100.0%)
		 matched with city = Dublin (100.0%)
query:3 of 1610 Country="UK" and City="London"
	 matched with countryID = GB (100.0%)
		 matched with city = London (100.0%)
query:4 of 1610 Country="United Kingdom" and City="London"
	 matched with countryID = GB (100.0%)
		 matched with city = London (100.0%)
query:5 of 1610 Country="UK" and City="London"
	 matched with countryID = GB (100.0%)
		 matched with city = London (100.0%)
query:6 of 1610 Country="Ireland" and City="Leixlip"
	 matched with countryID = IE (100.0%)
		 matched with city = Leixlip (100.0%)
query:7 of 1610 Country="Ireland" and City="Arklow"
	 matched with coun

As shown in the output above, the **COUNTRY** and **CITY** column entries can be evaluated with **townsDB** 

>if a match is found, then a **CountryKey** and **CityKey** are obtained. These keys allow the townsDB database to be indexed, and additional information to be linked such as **population** and l**ocation**:

In [0]:
for keys1,keys2 in zip(CountryKey,CityKey):
  cName = townsDB['TOWNS'][keys1]['TownName'][keys2]
  cPop = townsDB['TOWNS'][keys1]['Population'][keys2]
  if cPop == None: cPop = 'Unknown'
  cLocX = townsDB['TOWNS'][keys1]['Longitude'][keys2]
  cLocY = townsDB['TOWNS'][keys1]['Latitude'][keys2]
  print('countryKey: {}, cityKey: {}, cityName = {}  cityPopulation = {}, cityLocation = ({:.3f},{:.3f})'.format(keys1,keys2,cName,cPop,cLocX,cLocY))

countryKey: FR, cityKey: 25, cityName = Antibes  cityPopulation = 76393, cityLocation = (7.128,43.562)
countryKey: AE, cityKey: 2, cityName = Abu Dhabi  cityPopulation = Unknown, cityLocation = (54.367,24.467)
countryKey: IE, cityKey: 0, cityName = Dublin  cityPopulation = 1024027, cityLocation = (-6.249,53.333)
countryKey: GB, cityKey: 0, cityName = London  cityPopulation = 7556900, cityLocation = (-0.126,51.509)
countryKey: GB, cityKey: 0, cityName = London  cityPopulation = 7556900, cityLocation = (-0.126,51.509)
countryKey: GB, cityKey: 0, cityName = London  cityPopulation = 7556900, cityLocation = (-0.126,51.509)
countryKey: IE, cityKey: 134, cityName = Leixlip  cityPopulation = 15452, cityLocation = (-6.496,53.366)
countryKey: IE, cityKey: 38, cityName = Arklow  cityPopulation = 11761, cityLocation = (-6.141,52.793)
countryKey: IE, cityKey: 1, cityName = Cork  cityPopulation = 190384, cityLocation = (-8.471,51.898)
countryKey: IN, cityKey: 44, cityName = Ajmer  cityPopulation = 5

As demonsterated above, we can use the CountryKeys and CityKeys to extract and append more information to the original DataBase.

Below, is a printout of the information available for each placename. This information has been extracted from geoNames, and so any errors in the database originate from there. It is not garaunteed how accurate the values are, but GeoNames has a strong contributive community who are actively improving the database. 

In [0]:
townsDB['TOWNS']['IE'].keys()


dict_keys(['TownIndex', 'TownName', 'Country', 'Latitude', 'Longitude', 'Province', 'County', 'Address2', 'TownClass', 'Population', 'Aliases', 'AliasIndex'])